# Importing and Creating Sessions

In [1]:
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *

In [2]:
# Create a SparkSession
spark = SparkSession\
    .builder \
    .appName("Data Cleaning Process and Techniques") \
    .getOrCreate()

23/09/25 23:11:38 WARN Utils: Your hostname, FM-PC-LT-323 resolves to a loopback address: 127.0.1.1; using 192.168.18.19 instead (on interface wlp0s20f3)
23/09/25 23:11:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/25 23:11:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Reading CSv files

In [3]:
# Read the CSV file into a DataFrame
csv_file_path = 'us_home_data.csv'
home_df = spark.read.csv(csv_file_path, header=True, inferSchema=True)
home_df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- MS Sub Class: integer (nullable = true)
 |-- MS Zoning: string (nullable = true)
 |-- Lot Frontage: string (nullable = true)
 |-- Lot Area: integer (nullable = true)
 |-- Street: string (nullable = true)
 |-- Alley: string (nullable = true)
 |-- Lot Shape: string (nullable = true)
 |-- Land Contour: string (nullable = true)
 |-- Utilities: string (nullable = true)
 |-- Lot Config: string (nullable = true)
 |-- Land Slope: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Bl dg Type: string (nullable = true)
 |-- House Style: string (nullable = true)
 |-- Overall Qual: integer (nullable = true)
 |-- Overall Cond: integer (nullable = true)
 |-- Year Built: integer (nullable = true)
 |-- Year Remod Add: integer (nullable = true)
 |-- Roof Style: string (nullable = true)
 |-- Roof Matl: string (nullable = true)
 |-- Exterior First: string (nullable = true)
 |-- Exterior Second: string (nullable = true)
 |-- Mas Vnr Type: 

In [4]:
home_df.show(1000,truncate=False)

23/09/25 23:11:47 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+------------+---------+------------+--------+------+-----+---------+------------+---------+----------+----------+------------+----------+-----------+------------+------------+----------+--------------+----------+---------+--------------+---------------+------------+------------+----------+----------+----------+---------+---------+-------------+-------+----------+-----------+----------+-----------+--------------+--------------+---------+---------+--------------+--------------+------------+---------------+----------+-----------+-------------+----------+-------------+-------------+-----------+-----------+-----------+-----------+-----------+------------+-------------+--------------+------------+---------+------+-----+------------+--------+-------+-------+---------+--------------+----------+
|Id  |MS Sub Class|MS Zoning|Lot Frontage|Lot Area|Street|Alley|Lot Shape|Land Contour|Utilities|Lot Config|Land Slope|Neighborhood|Bl dg Type|House Style|Overall Qual|Overall Cond|Year Built|Year

# Back up the dataset

In [ ]:
# Save the DataFrame as Parquet
parquet_file_path = "us_home_data"
home_df.write.parquet(parquet_file_path)

In [ ]:
# Save the DataFrame as Json
json_file_path = "us_home_data.json"
home_df.write.json(json_file_path)

# Data Cleaning Process

## Heading Renaming and replacing " " with "_", Capital letter with small letter, and removing special character form column names

In [13]:
import re
home_re_df = home_df
home_re_df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- MS Sub Class: integer (nullable = true)
 |-- MS Zoning: string (nullable = true)
 |-- Lot Frontage: string (nullable = true)
 |-- Lot Area: integer (nullable = true)
 |-- Street: string (nullable = true)
 |-- Alley: string (nullable = true)
 |-- Lot Shape: string (nullable = true)
 |-- Land Contour: string (nullable = true)
 |-- Utilities: string (nullable = true)
 |-- Lot Config: string (nullable = true)
 |-- Land Slope: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Bl dg Type: string (nullable = true)
 |-- House Style: string (nullable = true)
 |-- Overall Qual: integer (nullable = true)
 |-- Overall Cond: integer (nullable = true)
 |-- Year Built: integer (nullable = true)
 |-- Year Remod Add: integer (nullable = true)
 |-- Roof Style: string (nullable = true)
 |-- Roof Matl: string (nullable = true)
 |-- Exterior First: string (nullable = true)
 |-- Exterior Second: string (nullable = true)
 |-- Mas Vnr Type: 

In [14]:
# Define a function to clean a column name
def clean_column_name(col_name):
    # Replace spaces and special characters with underscores, convert to lowercase
    cleaned_name = re.sub(r'[^a-zA-Z0-9]', '_', col_name.lower())
    return cleaned_name

# Create a list of renamed columns with aliases
selected_columns = [F.col(clean_column_name(col_name)).alias(col_name) for col_name in home_re_df.columns]

# Select the DataFrame with cleaned column names
home_re_df = home_re_df.select(*selected_columns)

# Print the schema of the cleaned DataFrame
home_re_df.printSchema()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `ms_sub_class` cannot be resolved. Did you mean one of the following? [`MS Sub Class`, `Bsmt Qual`, `Electrical`, `Misc Val`, `Alley`].;
'Project [id#17 AS Id#571, 'ms_sub_class AS MS Sub Class#572, 'ms_zoning AS MS Zoning#573, 'lot_frontage AS Lot Frontage#574, 'lot_area AS Lot Area#575, street#22 AS Street#576, alley#23 AS Alley#577, 'lot_shape AS Lot Shape#578, 'land_contour AS Land Contour#579, utilities#26 AS Utilities#580, 'lot_config AS Lot Config#581, 'land_slope AS Land Slope#582, neighborhood#29 AS Neighborhood#583, 'bl_dg_type AS Bl dg Type#584, 'house_style AS House Style#585, 'overall_qual AS Overall Qual#586, 'overall_cond AS Overall Cond#587, 'year_built AS Year Built#588, 'year_remod_add AS Year Remod Add#589, 'roof_style AS Roof Style#590, 'roof_matl AS Roof Matl#591, 'exterior_first AS Exterior First#592, 'exterior_second AS Exterior Second#593, 'mas_vnr_type AS Mas Vnr Typ

In [15]:
# Define a function to clean a column name
def clean_column_name(col_name):
    # Replace spaces and special characters with underscores, convert to lowercase
    cleaned_name = F.lower(F.regexp_replace(col_name, "[^a-zA-Z0-9]", "_"))
    return cleaned_name

In [16]:
# Create a list of renamed columns
renamed_columns = [clean_column_name(col_name).alias(col_name) for col_name in home_df.columns]

# Select the DataFrame with cleaned column names
home_df = home_df.select(*renamed_columns)

In [17]:
home_df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- MS Sub Class: string (nullable = true)
 |-- MS Zoning: string (nullable = true)
 |-- Lot Frontage: string (nullable = true)
 |-- Lot Area: string (nullable = true)
 |-- Street: string (nullable = true)
 |-- Alley: string (nullable = true)
 |-- Lot Shape: string (nullable = true)
 |-- Land Contour: string (nullable = true)
 |-- Utilities: string (nullable = true)
 |-- Lot Config: string (nullable = true)
 |-- Land Slope: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Bl dg Type: string (nullable = true)
 |-- House Style: string (nullable = true)
 |-- Overall Qual: string (nullable = true)
 |-- Overall Cond: string (nullable = true)
 |-- Year Built: string (nullable = true)
 |-- Year Remod Add: string (nullable = true)
 |-- Roof Style: string (nullable = true)
 |-- Roof Matl: string (nullable = true)
 |-- Exterior First: string (nullable = true)
 |-- Exterior Second: string (nullable = true)
 |-- Mas Vnr Type: string 

## How many missing data points do we have?

In [ ]:
# get the number of missing data for a single column in the dataset
missing_values_count_for_every_column = home_df.select([F.sum(F.col(c).isNull().cast("int")).alias(c) for c in home_df.columns])
missing_values_count_for_every_column.show(truncate=False)

In [ ]:
# get the percent of data missing

# Calculate the total number of cells in the DataFrame
total_cells = home_df.count() * len(home_df.columns)

# Calculate the total number of missing values
missing_values_count = home_df.select([F.when(F.col(c).isNull(), 1).otherwise(0).alias(c) for c in home_df.columns])
total_missing_data = missing_values_count.agg(*[F.sum(F.col(c)).alias(c) for c in missing_values_count.columns]).collect()[0]

# Calculate the percentage of missing data
percent_missing_data = (sum(total_missing_data) / total_cells) * 100
percent_missing_data